<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: Lachin Naghashyar<br>
   **Student ID**: 98110179<br> <br>

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

## Preparing Data
I used the Real estate price prediction data set which was built for regression analysis and includes the date of purchase, house age, location, distance to nearest MRT station, and house price of unit area. We can take a look at our data below:

In [2]:
import pandas as pd

df = pd.read_csv('Real estate.csv')
df

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1
...,...,...,...,...,...,...,...,...
409,410,2013.000,13.7,4082.01500,0,24.94155,121.50381,15.4
410,411,2012.667,5.6,90.45606,9,24.97433,121.54310,50.0
411,412,2013.250,18.8,390.96960,7,24.97923,121.53986,40.6
412,413,2013.000,8.1,104.81010,5,24.96674,121.54067,52.5


As you can see, it consists of different features for $X$ and also has the price of unit area as its final output ($Y$). In Regression, we are searching for relationships among variables. For example here, we are looking for a function that maps $X_i$ features to the price.
We can separate the data into $X$ and $Y$ arrays:

In [3]:
X = df.iloc[:, 1:7].to_numpy()
Y = df.iloc[:, 7:].to_numpy()

## Split data into Train and Test sets:
we use about 80 percent in the training and the rest in test set.

In [4]:
training_examples = int(X.shape[0] * 0.8)
X_train = X[:training_examples,:]
Y_train = Y[:training_examples,:]
X_test = X[training_examples:,:]
Y_test = Y[training_examples:,:]

## Create a Model (using scikit-learn package)

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import some packages needed:

In [6]:
model = LinearRegression()

Next we do the training using the fit function

In [7]:
model.fit(X_train, Y_train)

LinearRegression()

## Make predictions on the test set

In [8]:
predictions = model.predict(X_test)
mean_squared_error(Y_test, predictions)

59.40924577639657

## Tune the hyper parameters

For the LinearRegression function in sklearn we can make use of the documentations:
class sklearn.linear_model.LinearRegression(*, fit_intercept=True, normalize='deprecated', copy_X=True, n_jobs=None, positive=False)
For example we can try normalizing the data (which can be done by setting the fit_intercept parameter to False):

In [9]:
model = LinearRegression(fit_intercept=False).fit(X_train, Y_train)
predictions = model.predict(X_test)
mean_squared_error(Y_test, predictions)

59.993134416189

As you can see, the error didn't change much by normalizing the data, and it even increased slightly.
It seems that LinearRegression model does not have that many parameters that can be tuned, however, We can use sklearn.linear_model.SGDRegressor, which will provide many possibilities for tuning hyperparameters.

## SGDRegressor
class sklearn.linear_model.SGDRegressor(loss='squared_error', *, penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, random_state=None, learning_rate='invscaling', eta0=0.01, power_t=0.25, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, warm_start=False, average=False)

Linear model fitted by minimizing a regularized empirical loss with SGD.

SGD stands for Stochastic Gradient Descent: the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule (aka learning rate).

The regularizer is a penalty added to the loss function that shrinks model parameters towards the zero vector using either the squared euclidean norm L2 or the absolute norm L1 or a combination of both (Elastic Net). If the parameter update crosses the 0.0 value because of the regularizer, the update is truncated to 0.0 to allow for learning sparse models and achieve online feature selection.

In [34]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
# Always scale the input. The most convenient way is to use a pipeline.
reg = make_pipeline(StandardScaler(), SGDRegressor())
reg.fit(X_train, Y_train.ravel())
mean_squared_error(Y_test.ravel(), reg.predict(X_test))

59.6629795635549

In SGDRegressor we have the penalty{‘l2’, ‘l1’, ‘elasticnet’}, default=’l2’ field. Choosing l1, l2 and elasticnet will result in a Lasso, Ridge and elasticnet cost models respectively.

In [35]:
reg = make_pipeline(StandardScaler(), SGDRegressor(penalty='elasticnet'))
reg.fit(X_train, Y_train.ravel())
mean_squared_error(Y_test.ravel(), reg.predict(X_test))

59.60589578716992

In [36]:
reg = make_pipeline(StandardScaler(), SGDRegressor(penalty='l1'))
reg.fit(X_train, Y_train.ravel())
mean_squared_error(Y_test.ravel(), reg.predict(X_test))

59.705241257862006

It seems that they perform similar to each other and elasticnet might be slightly better in this example. We know that Lasso will eliminate many features, and reduce overfitting in your linear model. Ridge will reduce the impact of features that are not important in predicting your y values. Elastic Net combines feature elimination from Lasso and feature coefficient reduction from the Ridge model to improve your model's predictions (medium.com). We can also tune other parameters such as learning rate.

In [38]:
reg = make_pipeline(StandardScaler(), SGDRegressor(penalty='elasticnet',learning_rate='constant'))
reg.fit(X_train, Y_train.ravel())
mean_squared_error(Y_test.ravel(), reg.predict(X_test))

60.15492728876293

In [39]:
reg = make_pipeline(StandardScaler(), SGDRegressor(penalty='elasticnet',learning_rate='optimal'))
reg.fit(X_train, Y_train.ravel())
mean_squared_error(Y_test.ravel(), reg.predict(X_test))

1.1730253299359679e+24

In [40]:
reg = make_pipeline(StandardScaler(), SGDRegressor(penalty='elasticnet',learning_rate='adaptive'))
reg.fit(X_train, Y_train.ravel())
mean_squared_error(Y_test.ravel(), reg.predict(X_test))

59.41602680373095

It seems that adaptive learning rate performs slightly better.

In [48]:
reg = make_pipeline(StandardScaler(), SGDRegressor(penalty='elasticnet',learning_rate='adaptive', max_iter=3000))
reg.fit(X_train, Y_train.ravel())
mean_squared_error(Y_test.ravel(), reg.predict(X_test))

59.46618545262922

In [57]:
reg = make_pipeline(StandardScaler(), SGDRegressor(penalty='elasticnet',learning_rate='adaptive', alpha=0.0000001))
reg.fit(X_train, Y_train.ravel())
mean_squared_error(Y_test.ravel(), reg.predict(X_test))

59.42554674310714

Moreover, it seems like altering the max_iteration and alpha numbers doesn't affect the result that much.